# Overview

This notebok outlines 


# Install project dependencies

In [46]:
!pip install -r ../requirements.txt

# Define project dependencies

In [47]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
%autoreload
import os
import sys
from typing import Any

import pandas as pd
from tqdm.notebook import tqdm
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    pipeline,
    Trainer,
    TrainingArguments
)
from datasets import load_dataset


curr_dir = os.getcwd()
while os.path.basename(curr_dir) != "candidate_ranking":
    curr_dir = os.path.dirname(curr_dir)
sys.path.append(curr_dir)

/Users/jordi/Documents/Profesional/Búsqueda de Trabajo/Barcelona 2024/DocPlanner/candidate_ranking


100%|███████████████████████████████████████████████████████████████████████████| 3001/3001 [00:00<00:00, 3907.05it/s]


In [49]:
from src.codiesp import (
    CODIESP,
    TEST
)
from src.error_injector import corrupt_text
from src.language_model import BioClinicalBert

# Define experiment parameters

In [50]:
BIOBERT = BioClinicalBert()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
PATH_DATASET_TRAIN_SYNTHETIC = os.path.join(curr_dir, "data", "codiesp.synthetic.train.csv")
PATH_DATASET_TEST_SYNTHETIC = os.path.join(curr_dir, "data", "codiesp.synthetic.test.csv")

In [52]:
CODIESP.head()

,text_path,text_id,sentence
0,S1130-05582007000500007-1.txt,0,The patient is a 38-year-old male who attended...
1,S1130-05582007000500007-1.txt,0,1.
2,S1130-05582007000500007-1.txt,0,The pathological antecedents include being exa...
3,S1130-05582007000500007-1.txt,0,The examination revealed a right mandibular tu...
4,S1130-05582007000500007-1.txt,0,The orthopantomography showed a mixed lesion w...


# Define helper objects

## Data annotation functions `positive_case`, `negative_case`, `zero`

In [53]:
def positive_case(x: Any) -> int:
    return 1

def negative_case(x: Any) -> int:
    return 0

def zero(x: Any) -> int:
    return 0

# Synthesize train and test datasets by injecting simulated transcription errors

We’ll write a function to:
Randomly introduce errors (typos, swaps, deletions, insertions).
Prioritize errors in medical entities if specified.
Ensure a user-defined ratio of altered words.

## Training set

In [54]:
dataset = CODIESP.copy()
corrupted_dataset = dataset.copy()

In [55]:
dataset["label"] = dataset["sentence"].apply(negative_case)
dataset["num_character_errors"] = dataset["sentence"].apply(zero)
dataset["num_word_errors"] = dataset["sentence"].apply(zero)

In [56]:
error_rows = list(corrupted_dataset["sentence"].apply(
    lambda x: corrupt_text(x, word_error_probability=0.2, character_error_probability=0.2)
))

In [57]:
error_columns_df = pd.DataFrame(error_rows, columns=["sentence", "num_character_errors", "num_word_errors"])

corrupted_dataset = pd.concat(
    [corrupted_dataset.drop("sentence", axis=1, inplace=True), error_columns_df],
    axis=1
)

error_columns_df

,sentence,num_character_errors,num_word_errors
0,The patient is a 38-year-old male who attended...,3,0
1,.,1,0
2,The pathological antecedents include being exa...,3,0
3,Thf examination revealed right mandibular tum...,5,1
4,The orthopantomography showed a mixed lesion w...,2,0
...,...,...,...
8027,"In the denuded area, an amniotic memtrane rgaf...",6,0
8028,During follow-up therc waq a progressive reepi...,5,0
8029,Thgree weeks after surgery htere was a regular...,5,0
8030,The VA of thxe left improved to 4/10.,1,1


In [58]:
corrupted_dataset["label"] = corrupted_dataset.sentence.apply(positive_case)

# Merge and shuffle dataset
final_dataset = pd.concat([dataset, corrupted_dataset])
final_dataset.to_csv(PATH_DATASET_TRAIN_SYNTHETIC, index=False)

In [59]:
corrupted_dataset

,sentence,num_character_errors,num_word_errors,label
0,The patient is a 38-year-old male who attended...,3,0,1
1,.,1,0,1
2,The pathological antecedents include being exa...,3,0,1
3,Thf examination revealed right mandibular tum...,5,1,1
4,The orthopantomography showed a mixed lesion w...,2,0,1
...,...,...,...,...
8027,"In the denuded area, an amniotic memtrane rgaf...",6,0,1
8028,During follow-up therc waq a progressive reepi...,5,0,1
8029,Thgree weeks after surgery htere was a regular...,5,0,1
8030,The VA of thxe left improved to 4/10.,1,1,1


Explanation:
1. Medical terms from CANTEMIST are identified and corrupted preferentially based on entity_error_prob.
2. Non-medical words are corrupted with a lower probability other_word_prob.
3. The output dataset (corrupted_CANTEMIST.csv) contains:
   1. Original sentences (label = 0)
   2. Corrupted sentences (label = 1)

## Test set

We apply the same sequence of steps to add positive cases (errors) to the label set:

In [60]:
test = TEST.copy()
corrupted_test = test.copy()

test["label"] = test["sentence"].apply(negative_case)

error_rows_test = list(corrupted_test["sentence"].apply(
    lambda x: corrupt_text(x, word_error_probability=0.2, character_error_probability=0.2)
))

error_columns_test_df = pd.DataFrame(
    error_rows_test,
    columns=["sentence", "num_character_errors", "num_word_errors"]
)

corrupted_test = pd.concat(
    [corrupted_test.drop("sentence", axis=1, inplace=True), error_columns_test_df],
    axis=1
)

error_columns_test_df

corrupted_test["label"] = corrupted_dataset.sentence.apply(positive_case)

# Merge and shuffle dataset
final_test = pd.concat([test, corrupted_test])
final_test.to_csv(PATH_DATASET_TEST_SYNTHETIC, index=False)

# Training ClinicalBERT to Detect Errors

Now, we’ll train ClinicalBERT on the synthetic dataset to detect transcription errors.

In [61]:
# Load ClinicalBERT


# Load dataset
dataset = load_dataset("csv", data_files={"train": PATH_DATASET_SYNTHETIC})
dataset = dataset["train"].train_test_split(test_size=0.2)

print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text_path', 'text_id', 'sentence', 'label', 'num_character_errors', 'num_word_errors'],
        num_rows: 12851
    })
    test: Dataset({
        features: ['text_path', 'text_id', 'sentence', 'label', 'num_character_errors', 'num_word_errors'],
        num_rows: 3213
    })
})


In [ ]:
tokenized_datasets = dataset.map(BIOBERT.tokenize, batched=True)

# Training settings
training_args = TrainingArguments(
    output_dir="./error_detection",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
)

trainer = Trainer(
    model=BIOBERT.model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

# Train model
trainer.train()

Map:   0%|          | 0/12851 [00:00<?, ? examples/s]

Map:   0%|          | 0/3213 [00:00<?, ? examples/s]

/Users/jordi/Documents/Profesional/Búsqueda de Trabajo/Barcelona 2024/ADP/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


# Proof of concept: error Detection on new transcriptions

Once trained, ClinicalBERT can now classify new transcriptions for errors.

In [ ]:
error_detector = pipeline("text-classification", model=BIOBERT.model, tokenizer=BIOBERT.tokenizer)

test_text = "The patient has dyspnaea and needs immdeiate attention."
score = error_detector(test_text)
print(score)  # [{'label': '1' (error), 'score': 0.95}]


In [ ]:
test_text = "The patient has dyspnea and needs immediate attention."
score = error_detector(test_text)
score

- A high probability means the model detects errors.
- A low probability means the transcription is likely correct.

# Quantitative evaluation

In [ ]:
test["prediction"] = test["sentence"].apply(error_detector)['label']

In [ ]:
test[["label", "prediction"]].corr()

# Next Steps

1. Evaluate performance: Compute F1-score, precision, recall to ensure good error detection.
2. Refine error generation: Introduce more realistic mistakes (homophones, phonetic errors).
3. Fine-tune for specific error types: Add subcategories like misrecognized words vs. omitted words.